In [11]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import torch
df = pd.read_csv("/content/gdrive/MyDrive/tidydata.csv")
df['label'] = df['recommended'].apply(lambda x: 0 if x== True else 1)
X = df[['review']]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(df.index.values, df.label.values, test_size=0.2, random_state=40, stratify=df.label.values)
df['type'] = ['tmp']*df.shape[0]
df.loc[X_train, 'type'] = 'train'
df.loc[X_test, 'type'] = 'test'
X_train_list = list(df[df.type=='train'].review.values)
Y_train_list = list(df[df.type=='train'].label.values)
tmp1 = []
tmp2 = []
for i in range(len(X_train_list)):
    if X_train_list[i]==X_train_list[i]:
        tmp1.append(X_train_list[i])
        tmp2.append(Y_train_list[i])
X_train_list = tmp1
Y_train_list = tmp2

X_test_list = list(df[df.type=='test'].review.values)
Y_test_list = list(df[df.type=='test'].label.values)
tmp1 = []
tmp2 = []
for i in range(len(X_test_list)):
    if X_test_list[i]==X_test_list[i]:
        tmp1.append(X_test_list[i])
        tmp2.append(Y_test_list[i])
X_test_list = tmp1
Y_test_list = tmp2

In [13]:
from nltk.corpus import stopwords
import nltk
import re
nltk.download("stopwords")
stopwords_set = set(stopwords.words('english'))

sentences = []
for single_des in df[df.type=='train'].review.values:
    for s in re.split("\.", str(single_des)):
        if len(s) > 2:
            sentence = []
            for word in s.split(" "):
                if len(word)>1 and word not in stopwords_set:
                    sentence.append(word.strip().lower())
            sentences.append(sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
pip install gensim --upgrade

In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import gensim
from gensim.models import Word2Vec
import re
VECTOR_SIZE = 300
model = Word2Vec(sentences, vector_size=VECTOR_SIZE, window=15, min_count=5, workers=4)

In [16]:
words_vob = list(model.wv.index_to_key)
w2v_vector_train = np.zeros((len(X_train_list), VECTOR_SIZE))
for i in range(len(X_train_list)):
    word_list = X_train_list[i].split(" ")
    single_vector = np.zeros(VECTOR_SIZE)
    cnt = 0
    for word in word_list:
        word = word.strip().lower()
        if word in words_vob:
            single_vector += model.wv[word]
            cnt += 1
    if cnt > 0:
        w2v_vector_train[i] = single_vector / cnt

In [17]:
words_vob = list(model.wv.index_to_key)
w2v_vector_test = np.zeros((len(X_test_list), VECTOR_SIZE))
for i in range(len(X_test_list)):
    word_list = X_test_list[i].split(" ")
    single_vector = np.zeros(VECTOR_SIZE)
    cnt = 0
    for word in word_list:
        word = word.strip().lower()
        if word in words_vob:
            single_vector += model.wv[word]
            cnt += 1
    if cnt > 0:
        w2v_vector_test[i] = single_vector / cnt


In [18]:
train_emb = []
test_emb = []
for c in w2v_vector_train:
  train_emb.append(list(c))
for c in w2v_vector_test:
  test_emb.append(list(c))

In [19]:
import json
with open("/content/gdrive/MyDrive/df_train_w2v_win15.json", "w") as f:
    json.dump(train_emb, f)
with open("/content/gdrive/MyDrive/df_test_w2v_win15.json", "w") as f:
    json.dump(test_emb, f)